## basic_linear_regression: 
A simple single layer pytorch model to show how ```torch.nn.Linear``` solves general multi-variate linear regression problems.  
(_Tested using python3_)

___
### Idea:
The pytorch nn.Linear class can solve general Linear Equations of the kind:  $$\mathbf{y} = \mathbf{x}\mathbf{A^\top} + b$$ where:
* $\mathbf{A}$ is a transformation matrix which represent coefficients of the equation, 
* $\mathbf{x}$ is a vector of inputs and 
* $\mathbf{y}$ is a vector that is the result of the linear transformation

We are given: 
* rows of $\mathbf{x}$ values in a matrix $\mathbf{X}$,
* rows of $\mathbf{y}$ values in a matrix $\mathbf{Y}$

The model <span style="color:blue">SingleLayerNet</span> solves for the transformation matrix $\mathbf{A}$.

___
### Use:
##### In section 1.0
* set the scalar variable ```number_of_coefficients```
 * this variable determines the number of columns in Matrix $\mathbf{A}$
* set the scalar variable ```y_dimension```
 * the y_dimension determines:
   * the size of the $\mathbf{y}$ vector output of the linear transformation
   * this variable determines the number of rows in Matrix $\mathbf{A}$
* set the scalar bias term
* a scalar, (e.g. 0.1) in order to add some normally distributed random noise to the linear transformation

```
# exzmple from section 1.0
number_of_coefficients=3
y_dimension=number_of_coefficients
bias = 1
noise_level = 0.1
```

##### In section 2.0:
* run all cells from 2.0 on 

##### In section 3.0:
* run a single $\mathbf{x}$ vector test to see if the model's coefficients work
___

___
## 1.0 - Set the values below to determine: 
* the shape of matrix $\mathbf{A}$
* the shape of the output vector $\mathbf{y}$
* the amount of bias (the b term of the above linear equation)

In [ ]:
number_of_coefficients=3
y_dimension=number_of_coefficients
bias = 1
noise_level = 0.1

___
## 2.0  - Run all cells below to determine the values in the matrix $\mathbf{A}$

#### 2.01 Imports

In [ ]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import os,sys
import pdb
import torch 
from torch import nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from textwrap import wrap

#### 2.02 ```create_x_values``` creates a single row of training data

In [ ]:
# this method builds test y values/vectors for training
def create_x_values(x_vector):
    m = number_of_coefficients
    A = np.linspace(1,m,m).repeat(m).reshape(-1,m)
    return np.matmul(A,x_vector) + bias + np.random.randn() * noise_level

#### 2.03 Define the SingleLayerNet model

In [ ]:
# main model class
class SingleLayerNet(nn.Module):
  def __init__(self, D_in, D_out):
    super(SingleLayerNet, self).__init__()
    self.linear1 = nn.Linear(D_in, D_out) 
  def forward(self, x):
    return self.linear1(x)


In [ ]:
# number of training rows
n=10000
# number of elements in each batch on each epoch
b=100
# number of epochs
epochs=7000
# instantiate model
m1 = SingleLayerNet(number_of_coefficients,y_dimension)
# Create input torch Variables for X and Y
Xnp = np.random.rand(n,number_of_coefficients)
X = Variable(torch.Tensor(Xnp))
Ynp = np.array([create_x_values(x) for x in X])
Y = Variable(torch.Tensor(Ynp).reshape(-1,number_of_coefficients))

# create loss and optimizer
loss_fn = torch.nn.MSELoss(size_average = False) 
optimizer = optim.Adam(m1.parameters(), lr = 0.01)

# Training loop
for i in range(epochs):
    # create a batch of x values and y values (labels)
    indices = list(range(n))
    np.random.shuffle(indices)
    xb = X[indices[:b]]    
    yb = Y[indices][:b]
    # zero the optimizer
    optimizer.zero_grad()  # clear previous gradients
    
    # execute the forward pass to compute y values from equation xA^T + b (the linear transformation)
    output_batch = m1(xb)           # compute model output
    
    # calculate a loss
    loss = loss_fn(output_batch, yb)  # calculate loss

    # compute gradients
    loss.backward()        # compute gradients of all variables wrt loss
    optimizer.step()       # perform updates using calculated gradients
    # print out progress
    if i % 500 == 0 :
        print('epoch {}, loss {}'.format(i,loss.data))

# print model results
model_A = m1.linear1.weight.data.numpy()
model_bias = m1.linear1.bias.data.numpy()
print(f'A = {model_A}')
print(f'bias = {model_bias}')

___
## 3.0 run a test on a single x vector

In [ ]:
example_x = np.linspace(1,number_of_coefficients,number_of_coefficients)
print(f'example x value {example_x}')
example_y = create_x_values(example_x)
print(f'example y value to predict {example_y}')
example_prediction = m1(torch.Tensor(example_x.reshape(-1,number_of_coefficients))).data.numpy()
print(f'example y prediction {example_prediction}')
diffs = example_y - example_prediction
print(f'difference =  {diffs.round(4)}')

___
## 4.0 Now Run a toy example of finding the best hedge ratios for a portfolio of SP 500 Sector spdr ETF's vs the SPY ETF
* sector spdr symbols are 'XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLY', 'XLI', 'XLC', 'XLV', 'XLF'

### 4.1 Build and run model

In [ ]:
# get actual historical data for a small number of days (70)
df = pd.read_csv('./spdr_history.csv')
num_of_test_days = int(len(df) * .1)
df.values.shape
Ynp = df.iloc[:-num_of_test_days]['SPY'].values[:-num_of_test_days]
x_cols = sorted(['XLE', 'XLU', 'XLK', 'XLB', 'XLP', 'XLY', 'XLI', 'XLC', 'XLV', 'XLF'])
Xnp = df.iloc[:-num_of_test_days][x_cols].values[:-num_of_test_days]
print(Xnp.shape,Ynp.shape)# number of training rows
b=1
# number of epochs
epochs=10000
# instantiate model
m1 = SingleLayerNet(Xnp.shape[1],1)
# Create input torch Variables for X and Y
X = Variable(torch.Tensor(Xnp))
Y = Variable(torch.Tensor(Ynp).reshape(-1,1))

# create loss and optimizer
loss_fn = torch.nn.MSELoss(size_average = False) 
optimizer = optim.Adam(m1.parameters(), lr = 0.01)

# Training loop
for i in range(epochs):
    # create a batch of x values and y values (labels)
    indices = list(range(Xnp.shape[0]))
    np.random.shuffle(indices)
    xb = X[indices[:b]]    
    yb = Y[indices][:b]
    # zero the optimizer
    optimizer.zero_grad()  # clear previous gradients
    
    # execute the forward pass to compute y values from equation xA^T + b (the linear transformation)
    output_batch = m1(xb)           # compute model output
    
    # calculate a loss
    loss = loss_fn(output_batch, yb)  # calculate loss

    # compute gradients
    loss.backward()        # compute gradients of all variables wrt loss
    optimizer.step()       # perform updates using calculated gradients
    # print out progress
    if i % 500 == 0 :
        print('epoch {}, loss {}'.format(i,loss.data))

# print model results
model_A = m1.linear1.weight.data.numpy()
model_bias = m1.linear1.bias.data.numpy()
print(f'A = {model_A}')
print(f'bias = {model_bias}')

### Print hedge ratios

In [ ]:
df_output = pd.DataFrame({'symbol':x_cols,'hedge_ratio':model_A[0]})
df_output[['symbol','hedge_ratio']]

### Plot real y values vs simulated y values

In [ ]:
yreal = df['SPY'].values.reshape(-1)
all_Xnp = df[x_cols].values.reshape(-1,len(x_cols))
ysim = np.array(all_Xnp @ np.array(model_A[0]) + model_bias[0])
plot_with_pandas = False

# plot with pandas
if plot_with_pandas:
    df_results = pd.DataFrame({'yreal':yreal,'ysim':ysim})
    ax = df_results.plot(figsize = (16,7))
    ax.grid()
else:
    # plot with without pandas
    x_train = list(range(len(all_Xnp)))[:-num_of_test_days]
    x_test =  list(range(len(all_Xnp)))[-num_of_test_days-1:]
    ysim_train = ysim[:-num_of_test_days]
    ysim_test = ysim[-num_of_test_days-1:]
    yreal_train = yreal[:-num_of_test_days]
    yreal_test = yreal[-num_of_test_days-1:]
    fig, ax = plt.subplots(figsize = (16,7))

    ax.plot(x_train,yreal_train,color='blue',label='y_train_real')
    ax.plot(x_train,ysim_train,color='orange',label='y_train_model')
    ax.plot(x_test,yreal_test,color='red',label='y_test_real')
    ax.plot(x_test,ysim_test,color='green',label='y_test_model')
    ax.legend()
    ax.grid()
    hedge_ratio_dict = {x_cols[i]:model_A[0][i] for i in range(len(x_cols))}
    hr = {k:round(hedge_ratio_dict[k],4) for k in hedge_ratio_dict.keys()}
    t = f'SPY vs {hr}'
    t = t.replace("'","")
    title = ax.set_title("\n".join(wrap(t, 60)))
    fig.tight_layout()
    title.set_y(1.05)
    fig.subplots_adjust(top=0.8)


### End